In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import chi2
from numpy.linalg import multi_dot
from numpy.linalg import inv
from numpy.linalg import pinv
import matplotlib.pyplot as plt

In [2]:
# set format of the output
float_formatter = "{:.4f}".format # keep 4 digits
np.set_printoptions(formatter={'float_kind':float_formatter})

In [3]:
#******************************************************************************
# Define Paths + Read in Data + Clean Data
#******************************************************************************

# define paths of files 
path = os.getcwd() + "/Data"
FFFactors_path = '/factors/FFFactors.txt'
FF25_path = '/FF25/ff25vm.txt'
size10_path = '/size10/sizevm.txt'
industry10_path = '/industry10/ind10vm.txt'


# read in files
FFFactors = pd.read_csv(path + FFFactors_path, header = None, delim_whitespace=True)
FFFactors.columns = ['time', 'MKT-RF', 'SMB','HML', 'RF']
rmx = FFFactors['MKT-RF'].to_numpy().reshape(-1,1) # market excess return
rf = FFFactors['RF'].to_numpy().reshape(-1,1) # risk free rate
rm = rmx + rf #market return

FF25 = pd.read_csv(path + FF25_path, header = None, delim_whitespace=True)
size10 = pd.read_csv(path + size10_path, header = None, delim_whitespace=True)
industry10 = pd.read_csv(path + industry10_path, header = None, delim_whitespace=True)
# select test assets
# ta = 1: Fama-French 25 portfolios
# ta = 2: 10 size deciles
# ta = 3: 10 industry portfolios
# ta = 4: size and value factors - SMB and HML

ta = 1
if ta ==1:
    testass = FF25.iloc[:,1:26].to_numpy()
if ta ==2:
    testass = size10.iloc[:,10:20].to_numpy()
if ta ==3:
    testass = industry10.iloc[:,1:11].to_numpy()
if ta ==4:
    testass = FFFactors[['SMB','HML']].to_numpy()

# deal with missing values
testass[testass == -99.99] = 0
# check size of test assets: 25 portfolios


In [4]:
# whether using market as test asst or not
use_mkt = 1
capt = len(rf)
if use_mkt:
    print('USING MARKET AS TEST ASSET')
    testass = np.concatenate((rf, testass, rm), axis = 1)
else:
    print('NOT USING MARKET AS TEST ASSET')
    testass = np.hstack((rf, testass), axis = 1)

n = np.shape(testass)[1]

if ta<4:
    tax = testass - np.dot(rf, np.ones((1,n)))
else:
    tax = testass
    
tax = tax[:,1:]
nx = n - 1

fmat = np.concatenate((np.ones((capt,1)), rm), axis = 1)
fmat2 = fmat[:,1].reshape(-1,1)
fmx = np.concatenate((np.ones((capt,1)), rmx), axis = 1)
fmx2 = fmx[:,1].reshape(-1,1)

USING MARKET AS TEST ASSET


In [17]:
#*************************************************************;
#         EXCESS RETURNS    (p. 256 Cochrane)                 ;
#             m = 1 - b'f                                     ;
#*************************************************************;


# FIRST STAGE;
# %Think of as a cross-sectional regression of mean returns on 2nd moments;
d = np.dot(tax.transpose(), fmx2)/capt
d = d.reshape(-1, 1)
ep = np.mean(tax, axis = 0).transpose().reshape(-1, 1) # E_T[R_e]

b1 = inv(np.dot(d.transpose(), d))@d.transpose()@ep

errs = np.subtract(ep, np.dot(d, b1)) # E_T[R_e] - db, g_T(b)
bigerr = tax - np.multiply(tax, fmx2@b1) 

covmat = np.dot(bigerr.transpose(), bigerr)/capt #no temporal corr# S
covnoc = np.diag(np.diag(covmat))#iid
cov1l = covmat + (2/(capt-2)) * np.dot(bigerr[:-1,:].transpose(), bigerr[1:,:])#1l
cov4l = covmat # For how to choose mm, see NW(1994) "Automatic lag selection..." 
mm = 4
for i in range(1, mm+1):
    w = 1 - i/(mm+1)
    cov4l = cov4l + 2*w*(1/(capt-1-i))*np.dot(bigerr[:-i,:].transpose(), bigerr[i:,:])  

covb = (1/capt)*inv(np.dot(d.transpose(), d))*multi_dot([d.transpose(), covmat, d])*inv(np.dot(d.transpose(), d))
seb = np.sqrt(np.diag(covb))
idx = np.eye(nx)-d@inv(np.dot(d.transpose(), d))@d.transpose()

#no temporal corr
covmom = (1/capt)*idx@covmat@idx
chi2x = (errs.transpose()@pinv(covmom)@errs).item()
pval = 1 - chi2.cdf(chi2x, nx - np.size(d,1))

#iid
covmom_noc = (1/capt)*idx@covnoc@idx
chi2x_noc = (errs.transpose()@pinv(covmom_noc)@errs).item()
pval_noc = 1 - chi2.cdf(chi2x_noc, nx - np.size(d,1))

#1l
covmom_1l = (1/capt)*idx@cov1l@idx
chi2x_1l = (errs.transpose()@pinv(covmom_1l)@errs).item()
pval_1l = 1 - chi2.cdf(chi2x_1l, nx - np.size(d,1))

#4l
covmom_4l = (1/capt)*idx@cov4l@idx
chi2x_4l = (errs.transpose()@pinv(covmom_4l)@errs).item()
pval_4l = 1 - chi2.cdf(chi2x_4l, nx - np.size(d,1))


#ee = bigerr - np.multiply(np.ones((np.size(bigerr,0),1)), np.mean(bigerr, axis = 0))
#covmom2 = (1/(capt*capt))*np.dot(ee.transpose(), ee)
#chi22 = (errs.transpose()@pinv(covmom)@errs).item()
#pval2 = 1-chi2.cdf(chi22,nx -np.size(d,1))

chi2x = [chi2x_noc, chi2x, chi2x_1l, chi2x_4l]
pval = [pval_noc, pval, pval_1l, pval_4l]

print('\n\n\n EXCESS RETURNS, GMM FIRST STAGE \n\n');
#print(b1)
#print(seb)
print(chi2x)
print(pval)

#if use_mkt:
#    plt.plot(d[1:nx-2],ep[1:nx-2],'ko', d[nx-1],ep[nx-1],'kx', [0, 50], [0, b1.item(0,0)*50],'--')
#else:
#    plt.plot(d[1:nx],ep[1:nx],'ko', [0, 50], [0, b1.item(0,0)*50],'-');




 EXCESS RETURNS, GMM FIRST STAGE 


[15.562917026335354, 76.1852105022586, 71.1910197055599, 57.357060413617994]
[0.9272247053436178, 4.464203027243485e-07, 2.5567382497815316e-06, 0.00023939111003890012]


In [16]:
# SECOND STAGE ;
sinv = pinv(covmat)
sinv_noc = pinv(covnoc)
sinv_1l = pinv(cov1l)
sinv_4l = pinv(cov4l)

b2 = inv(d.transpose()@sinv@d)@d.transpose()@sinv@ep
covb = (1/capt)*inv(d.transpose()@sinv@d)
seb = np.sqrt(np.diag(covb))

chi2xx = (capt*errs.transpose()@sinv@errs).item()
pval = 1 - chi2.cdf(chi2xx,nx - np.size(d,1))

chi2xx_noc = (capt*errs.transpose()@sinv_noc@errs).item()
pval_noc = 1 - chi2.cdf(chi2xx_noc,nx - np.size(d,1))

chi2xx_1l = (capt*errs.transpose()@sinv_1l@errs).item()
pval_1l = 1 - chi2.cdf(chi2xx_1l,nx - np.size(d,1))

chi2xx_4l = (capt*errs.transpose()@sinv_4l@errs).item()
pval_4l = 1 - chi2.cdf(chi2xx_4l,nx - np.size(d,1))

print('\n\n\n\n SECOND STAGE \n\n')
#print(b2)
#print(seb)
print([chi2xx_noc, chi2xx, chi2xx_1l, chi2xx_4l])
print([pval_noc, pval, pval_1l, pval_4l])

#plt.plot([0, 50], [0, b2.item(0,0)*50],'-')
#plt.xlabel('E(Re*f)')
#plt.ylabel('E(Re)')
#if use_mkt:
#    plt.title('GMM / SDF Approach - TEST PORTFOLIOS and MKT')
#else:
#    plt.title('GMM / SDF Approach - TEST PORTFOLIOS ONLY')
#plt.show()





 SECOND STAGE 


[15.720437173066271, 76.19398852921564, 73.25640032016501, 57.40971601391183]
[0.9229000424215684, 4.4503438489940095e-07, 1.2496837910047631e-06, 0.0002355207894956246]
